In [1]:
# mou give me 32 thyroid annotations to train a model, which labeled by xiaobai's tool
# I will use python to parse to labelme's syntax.
#from 21/08/21

In [112]:
f1=r"/mnt/d/workspace/03-annotations/1.3.6.1.4.1.52026.66519049.5308.54-v1/1.3.6.1.4.1.52026.66519049.5308.55.1.1.json"

In [113]:
import json
with open(f1, 'r') as f:
    #data=f.read()
    j1 = json.load(f)

In [114]:
print(len(j1))
for i in j1:
    print(i)

5
PatientID
StudyID
data
series_instance_uid
sourceVIDEOFilePath


In [115]:
all_imgs_datas = j1['data']
print(len(all_imgs_datas), type(all_imgs_datas))
print(type(all_imgs_datas[0]))
j_data_items = all_imgs_datas[0]
for i in j_data_items:
    print(i)
print(j_data_items['index'])

245 <class 'list'>
<class 'dict'>
fileName
index
lesionCount
numberLayers
object
polygonCount
seriesNumber
size
sopUid
0


In [116]:
uids=[]
for data in all_imgs_datas:
    uids.append(int(data['sopUid'].split('.')[-1]))
uids.sort()
print(uids)

[0, 1, 2, 3, 4, 5, 6, 7, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235,

In [117]:
# get the absent annotations index
j = 0
for i in range(len(uids)):
    if i != uids[j]:
        print(i)
        #j=j-1
    else:
        j =j+1

8
9
10
11
12
13
14
15
16
34
35
36
37
38
39
40
41
42


In [118]:
j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

In [119]:
j_target_o = json.loads(j_target_s)

In [120]:
print(j_target_o)

{'shapes': [{'label': '颈动脉:CA', 'line_color': None, 'fill_color': None, 'points': [[296, 253], [295, 373], [430, 363], [427, 255]]}], 'lineColor': [0, 255, 0, 128], 'fillColor': [255, 0, 0, 128], 'imagePath': 'frm-0001.png', 'imageData': None}


In [123]:
import numpy
one_img = all_imgs_datas[0]
for item in one_img:
    print(item, type(item))
one_img_object = one_img['object']
print(type(one_img_object), len(one_img_object))

for img_data in all_imgs_datas:
    img_objs = img_data['object']
    n = int(img_data['sopUid'].split('.')[-1])
    print(f"img({n}) has {len(img_objs)} object.")
    polygonPoints = img_objs[0]['polygonPoint']
    print(f"polygonPoint length={len(polygonPoints)}")
    filename = f"frm-{(n+1):04d}"
    points_len = int((len(polygonPoints)))
    
    all_points = numpy.zeros((points_len, 2))
    all_points = all_points.tolist()
    print(len(all_points))
    for item in polygonPoints.items():
        #print("point item=", item, "len=", len(item))
        point_idx = int(item[0][1:])
        x0y1 = item[0][0]
        if 'x' == x0y1.lower():
            x0y1 = 0
        else:
            x0y1=1
        #print(f"point idx={point_idx}, coordinate={x0y1}")
        all_points[point_idx][x0y1] = int(item[1])
        #print(f">>>>>point[{point_idx}]=",all_points[int(point_idx)], ">>>>", all_points[point_idx][x0y1])
        #print(f"#all points={all_points[:20][:]}")
        #if (point_idx>10):
        #    break
    
    #print(f"all_points len={len(all_points)}")
    available_pts = []
    for pt in range(points_len):
        #print(f"point [{pt}]={all_points[pt]}")
        if all_points[pt][0] is None or all_points[pt][0]<10:
            #print(f"point[{pt}] is {all_points[pt]} delete it...")
            continue
        else:
            available_pts.append(all_points[pt])
            
    j_target_o['imagePath']=f"{filename}.png"
    j_target_o['shapes'][0]['label']='Thyroid'
    j_target_o['shapes'][0]['points']=available_pts
    
    with open(f"{filename}.json", 'w') as jfp:
        json.dump(j_target_o, jfp)
    
    #break

fileName <class 'str'>
index <class 'str'>
lesionCount <class 'str'>
numberLayers <class 'str'>
object <class 'str'>
polygonCount <class 'str'>
seriesNumber <class 'str'>
size <class 'str'>
sopUid <class 'str'>
<class 'list'> 1
img(0) has 1 object.
polygonPoint length=1438
1438
img(1) has 1 object.
polygonPoint length=1434
1434
img(100) has 1 object.
polygonPoint length=492
492
img(101) has 1 object.
polygonPoint length=472
472
img(102) has 1 object.
polygonPoint length=518
518
img(103) has 1 object.
polygonPoint length=542
542
img(104) has 1 object.
polygonPoint length=560
560
img(105) has 1 object.
polygonPoint length=552
552
img(106) has 1 object.
polygonPoint length=498
498
img(107) has 1 object.
polygonPoint length=532
532
img(108) has 1 object.
polygonPoint length=512
512
img(109) has 1 object.
polygonPoint length=544
544
img(110) has 1 object.
polygonPoint length=476
476
img(111) has 1 object.
polygonPoint length=642
642
img(112) has 1 object.
polygonPoint length=682
682
img(113

polygonPoint length=1140
1140
img(32) has 1 object.
polygonPoint length=1240
1240
img(33) has 1 object.
polygonPoint length=1144
1144
img(4) has 1 object.
polygonPoint length=1098
1098
img(43) has 1 object.
polygonPoint length=624
624
img(44) has 1 object.
polygonPoint length=548
548
img(45) has 1 object.
polygonPoint length=558
558
img(46) has 1 object.
polygonPoint length=524
524
img(47) has 1 object.
polygonPoint length=584
584
img(48) has 1 object.
polygonPoint length=610
610
img(49) has 1 object.
polygonPoint length=640
640
img(5) has 1 object.
polygonPoint length=1056
1056
img(50) has 1 object.
polygonPoint length=634
634
img(51) has 1 object.
polygonPoint length=544
544
img(52) has 1 object.
polygonPoint length=550
550
img(53) has 1 object.
polygonPoint length=524
524
img(54) has 1 object.
polygonPoint length=576
576
img(55) has 1 object.
polygonPoint length=672
672
img(56) has 1 object.
polygonPoint length=648
648
img(57) has 1 object.
polygonPoint length=656
656
img(58) has 1 

In [122]:
!pwd

/mnt/d/srcs/jupyter_notebook


In [146]:
# the full function to generate labelme json from xiaobai's json
import json 
import os

j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

def convert_to_labelme_format(xb_json_file, param):
    """
        deal with one xiaobai's json which have multi-labelme json
    """
    if not os.path.isfile(xb_json_file):
        print(f"ERR: json file:{xb_json_file} not valid")
        return -1
    with open(xb_json_file, 'r') as jfp:
        jdata = json.load(jfp)
    all_imgs_datas = jdata['data']
    series_instance_uid = jdata['series_instance_uid']
    #--01 create home directory
    target_json_home = xb_json_file.split('.json')[0] + '.dcm_frms'
    target_json_home = target_json_home.replace("03-annotations", "01-videos")
    os.makedirs(target_json_home, exist_ok=True)
    print(f"target_json_home={target_json_home}")
    #--02 extract json per image 
    for img_data in all_imgs_datas:
        img_objs = img_data['object']
        n = int(img_data['sopUid'].split('.')[-1])
        print(f"img({n}) has {len(img_objs)} object.")
        polygonPoints = img_objs[0]['polygonPoint']
        print(f"polygonPoint length={len(polygonPoints)}")
        filename = f"frm-{(n+1):04d}"
        points_len = int((len(polygonPoints)))

        all_points = numpy.zeros((points_len, 2))
        all_points = all_points.tolist()
        for item in polygonPoints.items():
            point_idx = int(item[0][1:])
            x0y1 = item[0][0]
            if 'x' == x0y1.lower():
                x0y1 = 0
            else:
                x0y1=1
            all_points[point_idx][x0y1] = int(item[1])
        #--03 delete zeros
        available_pts = []
        for pt in range(points_len):
            #print(f"point [{pt}]={all_points[pt]}")
            if all_points[pt][0] is None or all_points[pt][0]<10:
                #print(f"point[{pt}] is {all_points[pt]} delete it...")
                continue
            else:
                available_pts.append(all_points[pt])
        #--04 create target labelme format json
        j_target_o['imagePath']=f"{filename}.png"
        j_target_o['shapes'][0]['label']='Thyroid'
        j_target_o['shapes'][0]['points']=available_pts
        #--05 write to sidk
        target_json_name = os.path.join(target_json_home, f"{filename}.json")
        with open(target_json_name, 'w') as jfp:
            json.dump(j_target_o, jfp)
    
    

def convert_all_xiaobai_jsons(json_root_dir, count=0):
    if not os.path.exists(json_root_dir):
        print(f"Dir not exist:{json_root_dir}")
        return -1
    subdirs = []
    dirs = sorted(os.listdir(json_root_dir))
    for item in dirs:
        path = os.path.join(json_root_dir, item)
        if os.path.isdir(path):
            subdirs.append(path)
    #print(f"subdirs={subdirs}")
    
    folder_details = []
    json_f_count = 0
    for ct, json_dir in enumerate(subdirs):
        print("*"*32)
        print('Handle with {0} [{1}/]'.format(json_dir, ct + 1))
        files_in_dir = os.listdir(json_dir)
        if len(files_in_dir) < 2:
            getLogger().info(f"Empty dirctory : {json_dir}")
            continue
        else:
            for json_file in files_in_dir:
                fullfilename = os.path.join(json_dir, json_file)
                if not os.path.isfile(fullfilename):
                    continue
                #print("file in dir:", fullfilename)
                convert_to_labelme_format(fullfilename, 100)
                json_f_count = json_f_count+1
                if count != 0 and count == json_f_count:
                    print(f"\n<<<already extract {count} directory dcm file~")
                    return
    return 0

In [147]:
jhomes=r"/mnt/d/workspace/03-annotations/"
convert_all_xiaobai_jsons(jhomes, 0)

********************************
Handle with /mnt/d/workspace/03-annotations/1.3.6.1.4.1.52026.66519049.5308.54-v1 [1/]
target_json_home=/mnt/d/workspace/01-videos/1.3.6.1.4.1.52026.66519049.5308.54-v1/1.3.6.1.4.1.52026.66519049.5308.54.1.1.dcm_frms
img(0) has 1 object.
polygonPoint length=1202
img(1) has 1 object.
polygonPoint length=1202
img(10) has 1 object.
polygonPoint length=780
img(100) has 1 object.
polygonPoint length=736
img(101) has 1 object.
polygonPoint length=788
img(102) has 1 object.
polygonPoint length=734
img(103) has 1 object.
polygonPoint length=800
img(104) has 1 object.
polygonPoint length=854
img(105) has 1 object.
polygonPoint length=888
img(106) has 1 object.
polygonPoint length=894
img(107) has 1 object.
polygonPoint length=902
img(108) has 1 object.
polygonPoint length=904
img(109) has 1 object.
polygonPoint length=882
img(11) has 1 object.
polygonPoint length=748
img(110) has 1 object.
polygonPoint length=794
img(111) has 1 object.
polygonPoint length=756
im

img(46) has 1 object.
polygonPoint length=772
img(47) has 1 object.
polygonPoint length=772
img(48) has 1 object.
polygonPoint length=790
img(49) has 1 object.
polygonPoint length=768
img(5) has 1 object.
polygonPoint length=846
img(50) has 1 object.
polygonPoint length=776
img(51) has 1 object.
polygonPoint length=746
img(52) has 1 object.
polygonPoint length=758
img(53) has 1 object.
polygonPoint length=800
img(54) has 1 object.
polygonPoint length=788
img(55) has 1 object.
polygonPoint length=714
img(56) has 1 object.
polygonPoint length=724
img(57) has 1 object.
polygonPoint length=720
img(58) has 1 object.
polygonPoint length=736
img(59) has 1 object.
polygonPoint length=716
img(6) has 1 object.
polygonPoint length=850
img(60) has 1 object.
polygonPoint length=720
img(61) has 1 object.
polygonPoint length=740
img(62) has 1 object.
polygonPoint length=726
img(63) has 1 object.
polygonPoint length=688
img(64) has 1 object.
polygonPoint length=704
img(65) has 1 object.
polygonPoint l

img(219) has 1 object.
polygonPoint length=1126
img(22) has 1 object.
polygonPoint length=1232
img(220) has 1 object.
polygonPoint length=1126
img(221) has 1 object.
polygonPoint length=1114
img(222) has 1 object.
polygonPoint length=1056
img(223) has 1 object.
polygonPoint length=1056
img(224) has 1 object.
polygonPoint length=1056
img(225) has 2 object.
polygonPoint length=1052
img(226) has 1 object.
polygonPoint length=1050
img(23) has 1 object.
polygonPoint length=1176
img(231) has 1 object.
polygonPoint length=902
img(232) has 1 object.
polygonPoint length=898
img(233) has 1 object.
polygonPoint length=896
img(234) has 1 object.
polygonPoint length=902
img(235) has 1 object.
polygonPoint length=896
img(236) has 1 object.
polygonPoint length=916
img(237) has 1 object.
polygonPoint length=856
img(238) has 1 object.
polygonPoint length=854
img(239) has 1 object.
polygonPoint length=910
img(24) has 1 object.
polygonPoint length=1180
img(240) has 1 object.
polygonPoint length=870
img(2

img(98) has 1 object.
polygonPoint length=818
img(99) has 1 object.
polygonPoint length=912
target_json_home=/mnt/d/workspace/01-videos/1.3.6.1.4.1.52026.66519049.5308.54-v1/1.3.6.1.4.1.52026.66519049.5308.55.1.1.dcm_frms
img(0) has 1 object.
polygonPoint length=1438
img(1) has 1 object.
polygonPoint length=1434
img(100) has 1 object.
polygonPoint length=492
img(101) has 1 object.
polygonPoint length=472
img(102) has 1 object.
polygonPoint length=518
img(103) has 1 object.
polygonPoint length=542
img(104) has 1 object.
polygonPoint length=560
img(105) has 1 object.
polygonPoint length=552
img(106) has 1 object.
polygonPoint length=498
img(107) has 1 object.
polygonPoint length=532
img(108) has 1 object.
polygonPoint length=512
img(109) has 1 object.
polygonPoint length=544
img(110) has 1 object.
polygonPoint length=476
img(111) has 1 object.
polygonPoint length=642
img(112) has 1 object.
polygonPoint length=682
img(113) has 1 object.
polygonPoint length=658
img(114) has 1 object.
polyg

img(49) has 1 object.
polygonPoint length=640
img(5) has 1 object.
polygonPoint length=1056
img(50) has 1 object.
polygonPoint length=634
img(51) has 1 object.
polygonPoint length=544
img(52) has 1 object.
polygonPoint length=550
img(53) has 1 object.
polygonPoint length=524
img(54) has 1 object.
polygonPoint length=576
img(55) has 1 object.
polygonPoint length=672
img(56) has 1 object.
polygonPoint length=648
img(57) has 1 object.
polygonPoint length=656
img(58) has 1 object.
polygonPoint length=686
img(59) has 1 object.
polygonPoint length=674
img(6) has 1 object.
polygonPoint length=1052
img(60) has 1 object.
polygonPoint length=664
img(61) has 1 object.
polygonPoint length=648
img(62) has 1 object.
polygonPoint length=628
img(63) has 1 object.
polygonPoint length=616
img(64) has 1 object.
polygonPoint length=570
img(65) has 1 object.
polygonPoint length=536
img(66) has 1 object.
polygonPoint length=572
img(67) has 1 object.
polygonPoint length=530
img(68) has 1 object.
polygonPoint

...

img(193) has 1 object.
polygonPoint length=846
img(194) has 1 object.
polygonPoint length=712
img(195) has 1 object.
polygonPoint length=688
img(196) has 1 object.
polygonPoint length=670
img(197) has 1 object.
polygonPoint length=634
img(198) has 1 object.
polygonPoint length=646
img(199) has 1 object.
polygonPoint length=644
img(2) has 1 object.
polygonPoint length=848
img(211) has 1 object.
polygonPoint length=708
img(212) has 1 object.
polygonPoint length=712
img(213) has 1 object.
polygonPoint length=646
img(214) has 1 object.
polygonPoint length=646
img(25) has 1 object.
polygonPoint length=1046
img(26) has 1 object.
polygonPoint length=1046
img(27) has 1 object.
polygonPoint length=1046
img(28) has 1 object.
polygonPoint length=1008
img(29) has 1 object.
polygonPoint length=924
img(3) has 1 object.
polygonPoint length=654
img(30) has 1 object.
polygonPoint length=744
img(31) has 1 object.
polygonPoint length=738
img(32) has 1 object.
polygonPoint length=826
img(33) has 1 object.

img(56) has 1 object.
polygonPoint length=1190
img(77) has 1 object.
polygonPoint length=856
img(78) has 1 object.
polygonPoint length=886
img(79) has 1 object.
polygonPoint length=878
img(80) has 1 object.
polygonPoint length=852
img(81) has 1 object.
polygonPoint length=866
img(82) has 1 object.
polygonPoint length=870
img(83) has 1 object.
polygonPoint length=850
img(84) has 1 object.
polygonPoint length=878
img(85) has 1 object.
polygonPoint length=822
img(86) has 1 object.
polygonPoint length=820
img(87) has 1 object.
polygonPoint length=774
img(88) has 1 object.
polygonPoint length=750
img(9) has 1 object.
polygonPoint length=1558
img(90) has 1 object.
polygonPoint length=1002
img(91) has 1 object.
polygonPoint length=1360
img(92) has 1 object.
polygonPoint length=1492
img(93) has 1 object.
polygonPoint length=1492
img(94) has 1 object.
polygonPoint length=1512
img(95) has 1 object.
polygonPoint length=1828
img(96) has 1 object.
polygonPoint length=1828
img(97) has 1 object.
poly

img(67) has 1 object.
polygonPoint length=770
img(68) has 1 object.
polygonPoint length=768
img(69) has 1 object.
polygonPoint length=748
img(7) has 1 object.
polygonPoint length=1438
img(70) has 1 object.
polygonPoint length=824
img(71) has 1 object.
polygonPoint length=800
img(72) has 1 object.
polygonPoint length=832
img(73) has 1 object.
polygonPoint length=840
img(74) has 1 object.
polygonPoint length=818
img(75) has 1 object.
polygonPoint length=778
img(76) has 1 object.
polygonPoint length=778
img(77) has 1 object.
polygonPoint length=772
img(78) has 1 object.
polygonPoint length=738
img(79) has 1 object.
polygonPoint length=744
img(8) has 1 object.
polygonPoint length=1376
img(9) has 1 object.
polygonPoint length=1200
img(96) has 1 object.
polygonPoint length=920
img(97) has 1 object.
polygonPoint length=920
img(98) has 1 object.
polygonPoint length=972
img(99) has 1 object.
polygonPoint length=972
target_json_home=/mnt/d/workspace/01-videos/1.3.6.1.4.1.52026.66519049.5308.89-v

img(66) has 1 object.
polygonPoint length=712
img(67) has 1 object.
polygonPoint length=716
img(68) has 1 object.
polygonPoint length=712
img(69) has 1 object.
polygonPoint length=712
img(7) has 1 object.
polygonPoint length=824
img(70) has 1 object.
polygonPoint length=722
img(71) has 1 object.
polygonPoint length=712
img(72) has 1 object.
polygonPoint length=718
img(73) has 1 object.
polygonPoint length=750
img(74) has 1 object.
polygonPoint length=734
img(75) has 1 object.
polygonPoint length=814
img(76) has 1 object.
polygonPoint length=744
img(77) has 1 object.
polygonPoint length=734
img(78) has 1 object.
polygonPoint length=732
img(79) has 1 object.
polygonPoint length=732
img(8) has 1 object.
polygonPoint length=724
img(80) has 1 object.
polygonPoint length=722
img(81) has 1 object.
polygonPoint length=836
img(82) has 1 object.
polygonPoint length=776
img(83) has 1 object.
polygonPoint length=748
img(84) has 1 object.
polygonPoint length=768
img(85) has 1 object.
polygonPoint l

0